# Asg starter

## This is a more realistic calculator with registers where you can store values. Please be studying the code of this calculator well, as I may ask things about it in Midterm-2.

## Add the successor operator, represented by ```!```. Note that ```!``` is not factorial for us here.
## Examples:

* !3 = 4
* !!3 = 5
* 3 + 3 * !!!3 = 21
* 3 * !!3 + !!3 * !!!3 = 45

### In general, !expression obtains the successor of the expression

### You should generate a token "SUCC" corresponding to "!"

### Then modify p_expression_succ

### For your convenience, I've provided comments on where to add the changes

### (since you all don't know Python)

# Look for <== below !!


* Build your own calculator
  - The code is adapted slightly from https://www.dabeaz.com/ply/example.html

  - further seal knowledge of lexers (RE based) and parsers (CFG based) in your minds

# What to submit
* Run the notebook without any code additions, and you'll see a dialog box pop-up
* You can do things like 4+5 which will work
*  The dialog-box is ended by typing END as you can see in the test loop
* If you do !4, it will throw an error (Illegal character) as the lexer has no idea what ! means
* If you run the tests after the dialog box, most of those tests will fail
* Take a screenshot of THESE TESTS FAILING, i.e. the test after the "test_strings =..." definition
* Call this screenshot-1.png (say) and convert to screenshot-1.pdf

* ---

* Now do the code changes marked to add SUCC . Follow my comments
* Now if you run the code, it will calculate 4+5 as before, but also !4 will be computed as "5"
* You can end the dialog box by typing END
* Then all the succeding code will also run
* Run all the succeeding code
* Now all the tests after test_strings = ... will succeed
* Take a screenshot of these tests succeeding - say screenshot-2.png converted to screenshot-2.pdf

* ---

##   Submit the finished notebook after the second run + the above two screenshots


In [1]:
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True

if OWN_INSTALL:
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''
    sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',
                   '../../../..',  '../../../../3rdparty',
                   '../../..',     '../../../3rdparty',
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty' ]
else: # In colab
    ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
    sys.path.append('./Jove')
    sys.path.append('./Jove/jove')

# -- common imports --
from jove.lex import lex
from jove.yacc import yacc

Cloning into 'Jove'...
remote: Enumerating objects: 6709, done.
remote: Counting objects: 100% (2075/2075), done.
remote: Compressing objects: 100% (826/826), done.
remote: Total 6709 (delta 1265), reused 1941 (delta 1169), pack-reused 4634 (from 1)
Receiving objects: 100% (6709/6709), 10.59 MiB | 11.20 MiB/s, done.
Resolving deltas: 100% (4270/4270), done.


In [6]:
tokens = (
    'NAME','NUMBER',
    'PLUS','MINUS','TIMES','DIVIDE','EQUALS',
    'LPAREN','RPAREN','SUCC'                    # <== ADD the token 'SUCC'
    )

# Tokens

t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_EQUALS  = r'='
t_LPAREN  = r'\('
t_RPAREN  = r'\)'
t_NAME    = r'[a-zA-Z_][a-zA-Z0-9_]*'
t_SUCC    = r'!'                               # <== FIX this defn to generate token 'SUCC'

def t_NUMBER(t):
    r'\d+'
    try:
        t.value = int(t.value)
    except ValueError:
        print("Integer value too large %d", t.value)
        t.value = 0
    return t

# Ignored characters
t_ignore = " \t"

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count("\n")

def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)


# Parsing rules

precedence = (
    ('left','PLUS','MINUS'),
    ('left','TIMES','DIVIDE'),
    ('right','UMINUS' ),
    ('right','SUCC') #---                         <== add SUCC here with "right" for associativity
    )

# dictionary of names
names = { }

def p_statement_assign(t):
    'statement : NAME EQUALS expression'
    names[t[1]] = t[3]

def p_statement_expr(t):
    'statement : expression'
    print(t[1])

def p_expression_binop(t):
    '''expression : expression PLUS expression
                  | expression MINUS expression
                  | expression TIMES expression
                  | expression DIVIDE expression'''
    if t[2] == '+'  : t[0] = t[1] + t[3]
    elif t[2] == '-': t[0] = t[1] - t[3]
    elif t[2] == '*': t[0] = t[1] * t[3]
    elif t[2] == '/': t[0] = t[1] / t[3]

def p_expression_uminus(t):
    'expression : MINUS expression %prec UMINUS'
    t[0] = -t[2]

    #  def p_expression_succ(t): <=== DEFINE this function properly
    #                     <===== WRITE a suitable grammar rule to handle SUCC
    #                     <===== WRITE a suitable semantic action for SUCC

def p_expression_succ(t):
    'expression : SUCC expression'
    t[0] = t[2] + 1

def p_expression_group(t):
    'expression : LPAREN expression RPAREN'
    t[0] = t[2]

def p_expression_number(t):
    'expression : NUMBER'
    t[0] = t[1]

def p_expression_name(t):
    'expression : NAME'
    try:
        t[0] = names[t[1]]
    except LookupError:
        print("Undefined name '%s'" % t[1])
        t[0] = 0

def p_error(t):
    print("Syntax error at '%s'" % t.value)

calclexer = lex()
calcparser = yacc()

while True:
    try:
        s = input('calc > ')   # Use raw_input on Python 2
    except EOFError:
        break
    if (s=='END'):
        break
    calcparser.parse(s, lexer=calclexer)

Generating LALR tables


calc > !5
6
calc > !!5
7
calc > !!!11
14
calc > END


## If you implemented SUCC correctly, the test expression for SUCC (below) will work without errors.


In [7]:
calcparser.parse("3 * !!3 + !!3 * !!!3", lexer=calclexer)

45


## Your output must resemble the above.

In [8]:
test_strings = ['2+3', '2+!-3', '3 + 3 * -!!3 + !!3 * !!!-3', 'x=3', 'y=4', 'z=x+y', 'z', 'z=x+!y', 'z']

In [9]:
for s in test_strings:
    calcparser.parse(s, lexer=calclexer)

5
0
-12
7
8


# END